# Homework 4 - Spark

In this homework, we are practicing Apache Spark.

You are required to turn in this notebook as BDM\_HW4\_Spark\_**NetId**.ipynb. You will be asked to complete each task using Apache Spark. Output can be printed in the notebook.

## Task 1 (5 points)

You are asked to implement Homework 3 using Spark. The description is provided below for your convenience.

You are asked to implement the Social Triangle example discussed in class. In particular, given the email dataset, please list all "reciprocal" relationships in the company. Recall that:

If A emails B and B emails A, then A and B is *reciprocal*.

If A emails B but B doesn’t email A, then A and B is *directed*.

**Dataset:** We will use a subset of the open [Enron Email Dataset](https://www.cs.cmu.edu/~./enron/ "Enron Email Dataset"), which contains approximately 10,000 simplified email headers from the Enron Corporation. You can download this dataset from NYU Classes as **enron_mails_small.csv**. The file contains 3 columns *Date*, *From*, and *To*. Their description is as follows:

|Column name|Description|
|--|--|
|Date |The date and time of the email, in the format YYYY-MM-DD hh-mm-ss, <br />e.g. "1998-10-30 07:43:00" |
|From |The sender email address, <br />e.g. "mark.taylor@enron.com" |
|To | A list of recipients' email addresses separated by semicolons ';', <br />e.g. "jennifer.fraser@enron.com;jeffrey.hodge@enron.com" |

Note that, we only care about users employed by Enron, or only relationships having email addresses that end with *'@enron.com'*.

The expected output is also provided below. For each reciprocal relationship, please output a tuple consisting of two strings. The first one is always **'reciprocal'**. And the second one is a string showing the name of the two person in the following format: **'Jane Doe : John Doe'**. The names should be presented in the lexical order, i.e. there will not be a 'John Doe : Jane Doe' since 'Jane' is ordered before 'John.

Though the dataset only contains email addresses, not actual names, we're assuming that the email aliases were created based on their name. For example:

|Email Address|Converted Name|
|--|--|
|mark.taylor@enron.com|Mark Taylor|
|alan.aronowitz@enron.com|Alan Aronowitz|
|marc.r.cutler@enron.com|Marc R Cutler|
|hugh@enron.com|Hugh|

Please fill the code block with a series of MapReduce jobs using your own mapper and reducer functions. Be sure to include the naming convention logic into one of your mappers and/or reducers.

In [1]:
Emails = "enron_mails_small.csv"
emails = sc.textFile(Emails).cache()
list(enumerate(emails .first().split(',')))

[(0, 'Date'), (1, 'From'), (2, 'To')]

In [2]:
def pairs(partitionId,rows):
    if partitionId ==0:
        next(rows)
    import csv
    reader = csv.reader(rows)
    for row in reader:
        sender = row[1]
        if sender.endswith('@enron.com'):
            for recipient in row[2].split(';'):
                if recipient.endswith('@enron.com'):
                    if sender<recipient:
                        print(sender, recipient)
                        yield ((sender, recipient), 0)
                    else:
                        yield ((recipient, sender), 1)

def formatName(email):
    return email.split('@')[0].replace('.', ' ').title()

email_pairs = emails.mapPartitionsWithIndex(pairs)

In [3]:
output = email_pairs\
    .map(lambda x : (x[0],[x[1]]))\
    .reduceByKey(lambda x,y : x+y)\
    .filter(lambda x:len(set(x[1]))==2)\
    .map(lambda x:('reciprocal',formatName(x[0][0])+" : "+formatName(x[0][1])))

print(output.count())
output = sorted(output.collect())
output

35


[('reciprocal', 'Brenda Whitehead : Elizabeth Sager'),
 ('reciprocal', 'Carol Clair : Debra Perlingiere'),
 ('reciprocal', 'Carol Clair : Mark Taylor'),
 ('reciprocal', 'Carol Clair : Richard Sanders'),
 ('reciprocal', 'Carol Clair : Sara Shackleton'),
 ('reciprocal', 'Carol Clair : Tana Jones'),
 ('reciprocal', 'Debra Perlingiere : Kevin Ruscitti'),
 ('reciprocal', 'Drew Fossum : Susan Scott'),
 ('reciprocal', 'Elizabeth Sager : Janette Elbertson'),
 ('reciprocal', 'Elizabeth Sager : Mark Haedicke'),
 ('reciprocal', 'Elizabeth Sager : Mark Taylor'),
 ('reciprocal', 'Elizabeth Sager : Richard Sanders'),
 ('reciprocal', 'Eric Bass : Susan Scott'),
 ('reciprocal', 'Fletcher Sturm : Greg Whalley'),
 ('reciprocal', 'Fletcher Sturm : Sally Beck'),
 ('reciprocal', 'Gerald Nemec : Susan Scott'),
 ('reciprocal', 'Grant Masson : Vince Kaminski'),
 ('reciprocal', 'Greg Whalley : Richard Sanders'),
 ('reciprocal', 'Janette Elbertson : Mark Taylor'),
 ('reciprocal', 'Janette Elbertson : Richard Sa

## Task 2 (5 points)

You are asked to implement Task 2 of Lab 5. The description is provided below for your convenience.

We’ll be using two NYC open data sets: the SAT Results and the NYC High School Directory data sets. Both can be downloaded from the links below, or from online class resources.

**Dataset**: *Please note that each school is uniquely identified by an DBN code, which should be found on both data sets.*

**SAT_Results.csv**
Source: https://nycopendata.socrata.com/Education/SAT-Results/f9bf-2cp4  
Description: “The most recent school level results for New York City on the SAT. Results are available at the school level for the graduating seniors of 2012.”

**DOE_High_School_Directory_2014-2015.csv**
Source: https://data.cityofnewyork.us/Education/DOE-High-School-Directory-2014-2015/n3p6-zve2  
Description: “Directory of NYC High Schools.”

We would like to know how the Math scores vary across bus lines or subway lines serving the schools. Your task is to compute the average Math scores of all schools along each bus line and subway line. You can find the bus and subway lines serving each school in the High School Dictionary as bus and subway columns.

The expected results are two lists:
1. A list of key/value pairs: with bus line as keys, and the average Math scores as values.
2. A list of key/value pairs: with subway line as keys, and the average Math scores as values.

The top ten lines with highest score are shown below.

In [4]:
SAT_FN = "SAT_Results.csv"
HSD_FN = "DOE_High_School_Directory_2014-2015.csv"
sat = sc.textFile(SAT_FN).cache()
schools = sc.textFile(HSD_FN).cache()

In [5]:
def extractScores(partitionId,rows):
    if partitionId ==0:
        next(rows)
    import csv
    reader = csv.reader(rows)
    for fields in reader:
        if fields[4]!="s" and fields[2]!="s":
            yield (fields[0],(int(fields[4]),int(fields[2])))
satScores = sat.mapPartitionsWithIndex(extractScores)
satScores.take(5)

[('02M047', (400, 16)),
 ('21K410', (437, 475)),
 ('30Q301', (440, 98)),
 ('17K382', (374, 59)),
 ('18K637', (381, 35))]

In [6]:
def extractSchools_bus(partitionId,rows):
    if partitionId ==0:
        next(rows)
    import csv
    reader = csv.reader(rows)
    for fields in reader:
        if len(fields) == 58 and fields[17].isdigit():
            total_student = int(fields[17])
            for bus in fields[10].split(','):
                yield(fields[0],bus.strip())
    
Schools_bus = schools.mapPartitionsWithIndex(extractSchools_bus)
Schools_bus.take(5)

[('01M292', 'B39'),
 ('01M292', 'M14A'),
 ('01M292', 'M14D'),
 ('01M292', 'M15'),
 ('01M292', 'M15-SBS')]

In [7]:
def extractSchools_subway(partitionId,rows):
    if partitionId ==0:
        next(rows)
    import csv
    reader = csv.reader(rows)
    for fields in reader:
        if len(fields) == 58 and fields[17].isdigit():
            total_student = int(fields[17])
            for subways in fields[11].split(';'):
                for lines in subways.split(','):
                    if lines != "N/A":
                        lines = lines.strip()
                        yield(fields[0],lines[0])
    
Schools_subway = schools.mapPartitionsWithIndex(extractSchools_subway)
Schools_subway.take(5)

[('01M292', 'B'),
 ('01M292', 'D'),
 ('01M292', 'F'),
 ('01M292', 'J'),
 ('01M292', 'M')]

In [8]:
bus_line = Schools_bus.join(satScores).values() \
    .mapValues(lambda x :(x[0]*x[1],x[1]))\
    .reduceByKey(lambda x,y :(x[0]+y[0],x[1]+y[1])) \
    .mapValues(lambda x: int(x[0]/x[1]))
print("Bus Line:")
bus_line.sortBy(lambda x: -x[1]).take(10)

Bus Line:


[('S1115', 612),
 ('M79', 594),
 ('Q42', 582),
 ('M22', 574),
 ('Bx3', 571),
 ('B52', 560),
 ('B63', 557),
 ('B69', 548),
 ('B54', 543),
 ('B25', 541)]

In [9]:
subway_line = Schools_subway.join(satScores).values() \
    .mapValues(lambda x :(x[0]*x[1],x[1]))\
    .reduceByKey(lambda x,y :(x[0]+y[0],x[1]+y[1])) \
    .mapValues(lambda x: int(x[0]/x[1]))
print("Subway Line:")
subway_line.sortBy(lambda x: -x[1]).take(10)

Subway Line:


[('3', 513),
 ('A', 510),
 ('C', 510),
 ('R', 508),
 ('G', 503),
 ('D', 502),
 ('E', 501),
 ('1', 499),
 ('4', 495),
 ('N', 493)]